# Advent of Code 2015

## Day 7: Some Assembly Required

Solution code by [leechristie](https://github.com/leechristie) for Advent of Code 2015.

In [ ]:
from aoc import *

In [ ]:
from typing import Union
from abc import *

class uint16:

    __slots__ = ['value']

    def __init__(self, value: Union[int, 'uint16'] = 0):
        if type(value) == int:
            self.value = value
        else:
            assert type(value) == uint16
            self.value = value.value

    def __str__(self):
        return f'uint16({self.value})'

    def __repr__(self):
        return str(self)

    def op_not(self) -> 'uint16':
        return uint16(self.value ^ 65535)

    def op_and(self, other: Union[int, 'uint16']) -> 'uint16':
        if type(other) == uint16:
            other = other.value
        return uint16(self.value & other)

    def op_or(self, other: Union[int, 'uint16']) -> 'uint16':
        if type(other) == uint16:
            other = other.value
        return uint16(self.value | other)

    def op_lshift(self, other: Union[int, 'uint16']) -> 'uint16':
        if type(other) == uint16:
            other = other.value
        return uint16((self.value << other) & 65535)

    def op_rshift(self, other: Union[int, 'uint16']) -> 'uint16':
        if type(other) == uint16:
            other = other.value
        return uint16(self.value >> other)

In [ ]:
class Instruction(ABC):

    @staticmethod
    def validate_l_value(value: str) -> Union[str, int]:
        value = parse_int_or_str(value)
        if type(value) == str:
            assert is_non_empty_lowercase_alphabetic_only(value)
        else:
            assert type(value) == int
            value = uint16(value)
        return value

    @staticmethod
    def validate_binary_op_code(value: str) -> str:
        assert value in {'AND', 'RSHIFT', 'OR', 'LSHIFT', 'NOT'}
        return value

    @staticmethod
    def validate_unary_op_code(value: str) -> str:
        assert value in {'NOT'}
        return value

    @staticmethod
    def validate_r_value(value: str) -> str:
        assert is_non_empty_lowercase_alphabetic_only(value)
        return value

    @staticmethod
    def parse_instruction(tokens: tuple) -> 'Instruction':

        assert tokens[-2] == '->'

        if len(tokens) == 3:

            input_token, _, output_token = tokens

            input_token = Instruction.validate_l_value(input_token)
            output_token = Instruction.validate_r_value(output_token)

            return SetInstruction(input_token, output_token)

        elif len(tokens) == 5:

            left_token, op_token, right_token, _, output_token = tokens

            left_token = Instruction.validate_l_value(left_token)
            op_token = Instruction.validate_binary_op_code(op_token)
            right_token = Instruction.validate_l_value(right_token)
            output_token = Instruction.validate_r_value(output_token)

            return BinaryInstruction(left_token, op_token, right_token, output_token)

        elif len(tokens) == 4:

            op_token, right_token, _, output_token = tokens

            op_token = Instruction.validate_unary_op_code(op_token)
            right_token = Instruction.validate_l_value(right_token)
            output_token = Instruction.validate_r_value(output_token)

            return UnaryInstruction(op_token, right_token, output_token)

    @abstractmethod
    def wires(self) -> set[str]:
        pass

    @abstractmethod
    def evaluate(self, lookup: 'InstructionLookup') -> uint16:
        pass

    @abstractmethod
    def output_node_id(self) -> str:
        pass

    @abstractmethod
    def __str__(self):
        pass

    def __repr__(self):
        return str(self)

class SetInstruction(Instruction):

    def __init__(self, input_token: Union[str, uint16], output_token: str):
        self.input_token = input_token
        self.output_token = output_token
        self.memo = None

    def __str__(self):
        return f'SetInstruction({repr(self.input_token)}, {repr(self.output_token)})'

    def wires(self) -> set[str]:
        rv = set()
        for token in self.input_token, self.output_token:
            if type(token) == str:
                rv.add(token)
        return rv

    def evaluate(self, lookup: 'InstructionLookup') -> uint16:
        if self.memo is not None:
            return self.memo
        if type(self.input_token) == uint16:
            rv = self.input_token
        else:
            assert type(self.input_token) == str
            rv = lookup[self.input_token]
        self.memo = rv
        return rv

    def output_node_id(self) -> str:
        return self.output_token


class BinaryInstruction(Instruction):

    def __init__(self, left_token: Union[str, uint16], op_token: str, right_token: Union[str, uint16], output_token: str):
        self.left_token = left_token
        self.op_token = op_token
        self.right_token = right_token
        self.output_token = output_token
        self.memo = None

    def __str__(self):
        return f'BinaryInstruction({repr(self.left_token)}, {repr(self.op_token)}, {repr(self.right_token)}, {repr(self.output_token)})'

    def wires(self) -> set[str]:
        rv = set()
        for token in self.left_token, self.right_token, self.output_token:
            if type(token) == str:
                rv.add(token)
        return rv

    def evaluate(self, lookup: 'InstructionLookup') -> uint16:
        if self.memo is not None:
            return self.memo

        if type(self.left_token) == uint16:
            left_value = self.left_token
        else:
            assert type(self.left_token) == str
            left_value = lookup[self.left_token]

        if type(self.right_token) == uint16:
            right_value = self.right_token
        else:
            assert type(self.right_token) == str
            right_value = lookup[self.right_token]

        if self.op_token == 'AND':
            value = left_value.op_and(right_value)
        elif self.op_token == 'RSHIFT':
            value = left_value.op_rshift(right_value)
        elif self.op_token == 'OR':
            value = left_value.op_or(right_value)
        else:
            assert self.op_token == 'LSHIFT'
            value = left_value.op_lshift(right_value)

        self.memo = value
        return value

    def output_node_id(self) -> str:
        return self.output_token


class UnaryInstruction(Instruction):

    def __init__(self, op_token: str, right_token: Union[str, uint16], output_token: str):
        self.op_token = op_token
        self.right_token = right_token
        self.output_token = output_token
        self.memo = None

    def __str__(self):
        return f'UnaryInstruction({repr(self.op_token)}, {repr(self.right_token)}, {repr(self.output_token)})'

    def wires(self) -> set[str]:
        rv = set()
        for token in self.right_token, self.output_token:
            if type(token) == str:
                rv.add(token)
        return rv

    def evaluate(self, lookup: 'InstructionLookup') -> uint16:
        if self.memo is not None:
            return self.memo
        assert self.op_token == 'NOT'
        if type(self.right_token) == uint16:
            rv = self.right_token.op_not()
        else:
            assert type(self.right_token) == str
            rv = lookup[self.right_token].op_not()
        self.memo = rv
        return rv

    def output_node_id(self) -> str:
        return self.output_token

In [ ]:
class InstructionLookup:

    __slots__ = ['lookup']

    def __init__(self, instructions: list[Instruction]):
        self.lookup = {}
        for instruction in instructions:
            current_id = instruction.output_node_id()
            assert current_id not in self.lookup
            self.lookup[current_id] = instruction

    def __getitem__(self, item: str) -> uint16:
        return self.lookup[item].evaluate(self)

# Part 1

In [ ]:
def load_instructions(filename):
    INSTRUCTIONS = []
    for tokens in load_split_lines(filename, ' ', str):
        instruction = Instruction.parse_instruction(tokens)
        INSTRUCTIONS.append(instruction)
    return INSTRUCTIONS, InstructionLookup(INSTRUCTIONS)

In [ ]:
# load the instructions
INSTRUCTIONS, lookup = load_instructions('data/input07.txt')

In [ ]:
# get result from wire a
result_part1 = lookup['a'].value
print(f'The signal on wire a is {result_part1}')

### Part 2

In [ ]:
# reload
INSTRUCTIONS, lookup = load_instructions('data/input07.txt')

In [ ]:
# patch the wire
lookup.lookup['b'].input_token = uint16(result_part1)

In [ ]:
# get result from wire a
result_part2 = lookup['a'].value
print(f'The signal on wire a after patching wire b is {result_part2}')